In [1]:
from __future__ import division

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import viz
import cv2
import os
from os.path import isdir, join
from tqdm import tqdm


def rotate_body(points, yaw, pitch, roll):
    """
     Rotate the mesh based on the given degrees.
    yaw: +right, -left; pitch: +down,-up; roll: +counter clockwise, -clockwise

    """
    # the angles are all in degree
    rad_yaw = yaw*np.pi/180.0
    rad_pitch = pitch*np.pi/180.0
    rad_roll = roll*np.pi/180.0

    # This coord system differs from the theory, due to the orientation of basel mesh
    # We treat Z-axis as the X-axis
    R_roll = np.array([[np.cos(rad_roll),-np.sin(rad_roll),0], [np.sin(rad_roll),np.cos(rad_roll),0], [0,0,1]])
    R_yaw = np.array([[np.cos(rad_yaw),0,np.sin(rad_yaw)], [0,1,0], [-np.sin(rad_yaw),0,np.cos(rad_yaw)]])
    R_pitch = np.array([[1,0,0], [0,np.cos(rad_pitch),-np.sin(rad_pitch)], [0,np.sin(rad_pitch),np.cos(rad_pitch)]])

    rot_mat = np.identity(4)
    rot_mat[0:3,0:3] = R_yaw.dot(R_pitch).dot(R_roll)    
    # add homogeneous coords
    tmp = np.hstack((points,np.ones((points.shape[0],1))))
    new_points = tmp.dot(rot_mat.T)
    new_points = new_points[:,:3]
    
    return new_points

In [2]:
background = cv2.imread('test.jpg')

save_folder = 'results'
if not isdir(save_folder):
    os.makedirs(save_folder)

In [3]:
xyz_gt = np.load('xyz_coco.npy') # xyz,joint,frame  

# swap z and y axis for visualisation
xyz_gt = xyz_gt[...,[0,2,1]]
# flip z coord
xyz_gt[...,2] *= -1
# rotate the body to be upright
for i in range(xyz_gt.shape[0]):
    xyz_gt[i] = rotate_body(xyz_gt[i],0,-20,0)
    
frame_number = xyz_gt.shape[0]
    
# === Plot and animate ===
fig = plt.figure(figsize=(10,10))

ax = plt.gca(projection='3d')
ax.set_title('3D Skeleton Rendering',fontsize=30)

ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
ax.axes.zaxis.set_ticklabels([])
ax.set_xlabel("X",fontsize=30)
ax.set_ylabel("Y",fontsize=30)
ax.set_zlabel("Z",fontsize=30)

plt.grid(True)

fig.tight_layout()

# # visualise (96,) data: https://github.com/una-dinosauria/human-motion-prediction.git 
# ob = viz.Ax3DPose(ax)

# visualise Coco-format data (17,)
ob = viz.Ax3DPose17Point(ax)

# Plot the conditioning ground truth
for i in tqdm(range(frame_number)):        
    ob.update(xyz_gt[i])
    plt.show(block=False)
    fig.canvas.draw()
    # Now we can save it to a numpy array.
    render_image = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    
    render_image = render_image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
#     # resize image (if needed)
#     h, w = render_image.shape[:2]    
#     render_image = cv2.resize(render_image, (w//2,h//2))
    
    # fill the image with skeleton rendering ( put it on top left for now)    
    new_image = background.copy()
    new_image[:render_image.shape[0], :render_image.shape[1]] = render_image
    
    cv2.imwrite(join(save_folder,'{:03d}.jpg'.format(i)), new_image)        

100%|██████████| 1609/1609 [02:18<00:00, 12.18it/s]
